## Desafío 1

Grupo

Objetivos:
-  Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.
-  Realizar un análisis descriptivo de las principales variables.
-  Crear nuevas columnas a partir de las características dadas que puedan tener valor predictivo.

 



In [197]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/properatti.csv.gz', compression='gzip', header=0)
df.head(2)


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.0,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...


In [198]:
#Print columns
df.columns


Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [199]:
#Cantidad de registros, info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
Unnamed: 0                    121220 non-null int64
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 no

In [200]:
#Valores nulos por columna
df.isnull().sum().sort_values()

Unnamed: 0                         0
operation                          0
property_type                      0
place_with_parent_names            0
country_name                       0
state_name                         0
properati_url                      0
title                              0
description                        2
place_name                        23
image_thumbnail                 3112
geonames_id                    18717
surface_covered_in_m2          19907
price                          20410
price_aprox_local_currency     20410
price_aprox_usd                20410
currency                       20411
price_per_m2                   33562
surface_total_in_m2            39328
lat                            51550
lat-lon                        51550
lon                            51550
price_usd_per_m2               52603
rooms                          73830
expenses                      106958
floor                         113321
dtype: int64

Total registros: 121220

Valores necesarios: 

- place_name 

Barrio/3 nivel de ubicacion; 1er y 2do nivel está completo (country_name/state_name). Tenemos completo -place_with_parent_names, de ahi se puede extraer el 3er nivel
- surface_total_in_m2
- price
- price_per_m2
- rooms

# PLACE NAME

In [201]:

#Verifico la 3 columna de place_with_parent_names
df[df["place_name"].isnull()]["place_with_parent_names"].str.split('|').str[3]



6489     Tigre
10201    Tigre
11451    Tigre
14839    Tigre
18622    Tigre
21922    Tigre
23664    Tigre
24722    Tigre
38856    Tigre
45970    Tigre
46642    Tigre
53130    Tigre
55306    Tigre
57703    Tigre
57759    Tigre
57764    Tigre
57793    Tigre
58004    Tigre
58037    Tigre
59069    Tigre
62411    Tigre
62413    Tigre
63849    Tigre
Name: place_with_parent_names, dtype: object

In [202]:
df['place_name'].value_counts()

Córdoba                          9254
Rosario                          8504
Mar del Plata                    6534
Tigre                            3324
Nordelta                         3315
Belgrano                         2992
Palermo                          2885
La Plata                         2534
Caballito                        2273
Pilar                            2228
Villa Urquiza                    1632
Olivos                           1590
Recoleta                         1547
Lomas de Zamora                  1529
Flores                           1354
Ituzaingó                        1340
Villa Crespo                     1331
Capital Federal                  1297
San Telmo                        1216
Morón                            1209
Almagro                          1165
Barrio Norte                     1140
Ramos Mejía                      1127
Martínez                         1058
Lanús                            1048
Escobar                          1037
Banfield    

In [203]:
#Cargo los datos faltantes de place_name, creamos una nueva columna como flag y elimino la columna place_with_parent_names

df = df.assign(place_name_was_null=0)

df.loc[df["place_name"].isnull(),"place_name_was_null"] = 1;

df.loc[df["place_name"].isnull(),"place_name"] = df["place_with_parent_names"].apply(lambda place_with_parent_names: place_with_parent_names.split('|')[3])

df = df.drop(["place_with_parent_names"], axis=1)


In [206]:
df['place_name'].value_counts().head(10)

Córdoba          9254
Rosario          8504
Mar del Plata    6534
Tigre            3347
Nordelta         3315
Belgrano         2992
Palermo          2885
La Plata         2534
Caballito        2273
Pilar            2228
Name: place_name, dtype: int64

In [323]:
#Valores nulos por columna
df.isnull().sum().sort_values()

Unnamed: 0                         0
title                              0
properati_url                      0
place_name_was_null                0
state_name                         0
room_was_null                      0
operation                          0
property_type                      0
country_name                       0
place_name                         0
description                        2
image_thumbnail                 3007
geonames_id                    18300
surface_covered_in_m2          18995
price_aprox_usd                19420
price_aprox_local_currency     19420
price                          19420
currency                       19421
rooms                          28171
price_per_m2                   31943
surface_total_in_m2            38377
lon                            48427
lat                            48427
lat-lon                        48427
price_usd_per_m2               50872
expenses                      102814
floor                         108478
d

In [208]:
df['place_name_was_null'].value_counts()

0    121197
1        23
Name: place_name_was_null, dtype: int64

# Rooms

Para los casos donde no tenemos el valor "rooms", intentaremos sacar informacion de la columna descripcion. Creamos una nueva columna como flag para indicar si rooms no estaba cargado

In [209]:
df['rooms'].value_counts()

3.0     12567
2.0     12433
4.0      7937
1.0      7377
5.0      3782
6.0      1403
7.0       770
8.0       409
10.0      226
9.0       184
11.0       72
12.0       65
13.0       33
14.0       27
15.0       26
17.0       16
16.0       11
20.0       11
22.0        8
18.0        6
21.0        5
30.0        4
25.0        4
19.0        4
32.0        3
23.0        2
24.0        1
27.0        1
29.0        1
31.0        1
28.0        1
Name: rooms, dtype: int64

In [210]:
df = df.assign(room_was_null=0)
df.loc[df["rooms"].isnull(),"room_was_null"] = 1;
df['room_was_null'].value_counts()

1    73830
0    47390
Name: room_was_null, dtype: int64

In [304]:
regexp="(\d+)(\s|())amb"
df.loc[df['rooms'].isnull(),'rooms'] = df.loc[(df["rooms"].isnull()),"description"].str.lower().str.extract(regexp)[0]

(df['rooms'].isnull() & (df['description'].str.lower().str.contains('mono')),'rooms')[0].value_counts()

df.loc[(df['rooms'].isnull() & (df['description'].str.lower().str.contains('mono')),'rooms')[0]] = 1
#x Dormitorios
regexp="(\d+)(\s|())dorm"

df.loc[df['rooms'].isnull(),'rooms'] = df.loc[(df["rooms"].isnull()),"description"].str.lower().str.extract(regexp)[0]

# no especifica cantidad de dormitorios:
regexp="(\s|())dorm"
dormitorio


Aun asi quedan 30715 registros sin valor para rooms. Revisamos el valor "surface_covered_in_m2" para estimar los ambientes. 
surface_covered_in_m2 menor a 40mts2 - mono
entre 40 y 60 m2 - 2 amb
entre 60 y 80 m2 - 3 amb
80 y 100 m2 - 4 amb
100 y 120 - 5 amb


In [298]:
def calcula_ambientes_xmt2 (mts):
    
    try:
       val = int(mts)
    except ValueError:
        return np.nan; #print("That's not an int!. Value: " + str(mts))

    #print(str(str(mts) + str(type(mts) + "converted to int: "  +(val.str()))));
    val;

    if val <40:
       print("1");
       return 1;
    elif val <60:
       print("2");
       return 2;
    elif  val <80:
       print("3");
       return 3;
    elif  val <100:
       print("4");
       return 4;
    elif  val <120:
       print("5");
       return 5;
    elif  val >120:
       print("6");
       return 6;
    else:
       return np.nan;
  
        

In [322]:
df['description'][1086]#1


'Excelente Edificio Building, con 3 años de antigüedad. Departamento en altura al fente con excelente vista. Cuenta con living comedor al frente, cocina integrada equipada con muebles bajo y sobre mesada, baño completo con bañera y dormitorio con placard. La propiedad posee pisos de porcelanato, carpintería de alumino anodizado en linea módena, grifería fv, muebles de cocina marca   "Reno  ". Excelentes terminaciones. Apto Banco. Alquilado. Consulte'

In [319]:
#df.loc[df["rooms"].isnull(),"rooms"] = df["surface_covered_in_m2"].apply(lambda surface_covered_in_m2: calcula_ambientes_xmt2(surface_covered_in_m2))
#df["surface_total_in_m2"
#df.loc[df["rooms"].isnull() & df["surface_covered_in_m2"].isnull(),"rooms"] = df["rooms"].apply(calcula_ambientes_xmt2(df["surface_total_in_m2"]))

df.loc[df["rooms"].isnull() & df["surface_covered_in_m2"].isnull(),"rooms"] = df["surface_total_in_m2"].apply(lambda surface_total_in_m2: calcula_ambientes_xmt2(surface_total_in_m2))
#df["rooms"].apply(lambda surface_covered_in_m2: calcula_ambientes_xmt2(surface_covered_in_m2))

df.loc[df["rooms"].isnull() & df["surface_covered_in_m2"].isnull(),"surface_total_in_m2"]


2
2
1
2
5
1
3
1
6
1
2
2
1
6
6
6
5
6
5
6
2
6
6
6
6
6
2
6
6
2
6
6
4
6
3
6
6
5
2
1
6
1
6
4
6
6
6
6
2
5
4
4
6
1
6
3
2
6
6
6
6
1
6
6
6
1
4
6
6
6
2
1
2
2
3
6
3
2
6
1
1
1
1
6
6
6
6
6
1
6
6
3
6
6
6
6
6
6
6
6
6
4
6
6
6
4
6
5
6
6
2
5
4
6
1
4
1
1
6
6
1
1
1
1
6
2
2
4
6
6
2
2
6
3
6
6
6
6
5
1
1
2
2
3
2
6
1
2
2
4
6
2
1
6
3
3
3
6
6
3
2
6
6
1
6
4
1
6
6
6
1
6
6
6
2
1
6
2
5
1
4
1
1
1
1
4
6
6
1
1
6
6
6
6
6
1
6
6
6
2
6
1
3
3
2
4
6
2
5
3
6
6
6
6
6
2
6
6
2
6
1
6
6
2
6
6
6
6
6
3
6
6
4
6
1
6
6
6
3
4
4
5
6
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
6
6
2
2
2
1
6
2
2
5
2
1
3
6
3
6
6
2
3
6
6
6
6
3
6
6
2
2
6
2
3
2
6
6
6
6
6
3
6
2
6
6
6
6
6
6
6
6
4
2
2
6
2
6
6
6
6
1
6
6
6
2
6
6
6
3
3
1
6
1
3
2
6
6
3
2
1
5
4
1
2
4
6
6
6
6
6
6
6
2
6
6
6
2
6
6
6
3
6
6
6
6
6
6
1
1
1
1
1
1
1
1
1
1
3
3
6
1
6
6
1
1
1
1
5
6
5
5
6
2
6
4
4
3
5
2
2
3
3
2
3
6
6
3
2
6
6
6
2
3
3
6
6
3
6
4
3
2
1
6
6
4
2
6
2
6
3
6
5
5
6
6
2
6
5
3
2
2
3
3
3
2
2
2
2
2
2
2
1
3
6
2
4
2
6
2
2
6
2
2
1
6
1
6
3
6
2
3
2
2
1
2
2
2
2
1
6
3
1
2
6
6
4
6
3
1
3
6
4
3
6
6
6
4


3
6
5
5
3
5
6
1
2
3
3
2
1
6
6
2
6
2
3
6
6
6
6
1
1
3
3
1
1
1
2
1
6
2
4
6
3
1
6
1
1
1
2
6
2
6
2
2
2
6
6
6
2
6
3
2
2
2
3
6
3
2
6
6
5
6
3
6
6
2
1
6
2
6
5
3
6
6
2
6
6
1
6
6
6
2
6
6
6
1
4
6
6
1
2
6
4
2
6
1
6
6
6
6
2
5
3
6
6
6
3
2
6
6
5
3
4
2
6
6
6
3
6
6
6
1
2
1
4
4
3
2
6
2
5
3
6
4
6
6
6
6
6
6
6
6
6
4
4
6
6
6
1
2
2
1
3
6
3
3
2
3
6
1
6
6
4
3
2
6
6
4
2
6
6
6
6
6
2
2
2
6
1
1
3
6
5
4
5
5
4
2
4
4
5
5
5
5
6
6
6
2
4
6
1
3
6
6
6
6
2
6
2
6
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
6
6
6
6
6
6
5
1
3
3
5
4
5
1
5
4
1
5
6
5
6
6
5
4
5
4
6
5
4
6
1
1
2
6
1
3
2
6
2
2
3
2
1
6
6
5
3
6
3
6
2
1
4
4
6
6
6
1
3
2
5
3
3
5
6
5
6
3
1
2
6
6
2
1
6
2
2
6
5
6
2
1
2
1
2
2
6
2
3
2
6
3
6
6
3
6
3
6
6
6
2
6
6
6
4
1
4
6
1
1
2
3
6
2
2
2
6
2
3
2
4
2
4
2
6
6
5
3
4
6
6
1
6
2
1
6
2
1
2
3
4
6
6
6
3
1
2
1
6
4
6
6
2
6
1
1
1
1
3
6
2
1
6
5
4
4
6
3
6
6
6
4
2
3
3
5
3
4
3
2
4
1
6
6
4
4
4
6
5
6
2
4
1
2
2
1
6
6
6
6
6
6
6
6
2
6
3
6
3
1
4
6
6
1
1
6
3
6
6
2
6
2
1
4
5
6
4
6
2
6
2
4
6
5
2
3
6
6
5
1
1
1
1
1
4
5
6
2
1
6
6
6
2
6
2
1
1
6
6
6
6
6
4
6
6
1
4
6
6
4
1


6
6
1
4
4
6
2
4
6
6
1
6
4
3
2
6
6
2
1
3
5
5
2
6
2
3
6
6
6
5
6
6
6
6
3
6
5
6
6
6
6
6
6
6
2
2
6
6
6
1
1
6
1
4
5
6
6
6
6
6
3
6
6
3
2
6
6
2
6
1
6
4
6
2
3
6
1
5
2
1
1
1
1
1
1
6
6
5
6
6
2
3
6
1
1
6
6
6
6
3
6
6
6
6
6
6
6
2
6
3
6
3
5
6
6
6
6
6
6
6
6
6
2
6
6
6
6
6
6
6
6
1
3
6
2
6
6
6
2
6
6
6
6
3
2
2
1
6
6
3
6
6
5
1
1
2
3
6
2
3
4
6
4
2
3
6
2
6
6
6
6
2
5
2
6
6
2
2
6
5
6
6
2
1
4
4
5
1
3
2
2
4
1
1
1
6
6
2
5
6
4
2
5
6
3
6
5
6
3
6
2
1
6
6
6
6
6
1
3
3
1
6
6
6
3
3
6
6
6
1
2
2
6
4
6
2
6
1
2
5
6
6
1
2
6
4
6
6
2
3
6
6
6
6
6
6
6
2
3
3
3
6
6
1
6
6
6
6
2
6
6
1
6
6
6
6
6
6
2
4
6
6
4
2
3
1
6
1
1
6
3
2
2
4
2
1
6
6
2
2
4
2
2
2
6
6
2
1
1
6
6
2
3
6
6
3
2
2
2
2
6
6
3
1
3
3
6
6
2
6
6
6
2
2
2
3
3
6
4
6
6
6
6
6
6
6
6
6
6
6
3
3
3
6
2
6
6
4
6
4
4
6
6
6
6
6
6
6
2
6
2
6
6
1
2
1
2
1
1
4
4
1
1
6
4
6
4
1
4
3
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
2
5
6
6
2
3
6
4
6
5
6
6
6
6
6
6
1
4
6
6
6
6
5
6
3
6
4
2
3
2
2
6
6
1
6
6
3
4
6
6
6
3
6
3
5
4
6
3
6
6
2
6
6
6
6
6
1
1
1
3
6
2
6
2
1
6
6
3
6
5
6
4
6
6
2
4
4
2
6
4
3
2
6
1
1
2
6
6


6
6
2
1
1
1
1
2
2
2
4
6
4
4
6
6
4
6
2
5
4
3
6
6
6
6
6
1
6
2
3
6
6
4
1
6
6
2
6
6
6
1
1
1
4
6
1
1
1
5
1
6
6
1
4
1
6
2
6
1
3
3
5
6
6
2
1
4
3
6
6
3
6
4
6
6
6
6
6
6
6
6
5
2
6
3
2
2
5
4
4
4
4
4
4
6
4
6
2
6
6
6
3
6
6
6
1
2
1
3
4
6
2
2
2
3
6
1
6
6
6
6
1
3
4
6
5
6
5
6
6
3
1
2
4
3
6
1
6
6
4
6
6
1
5
1
6
3
6
6
3
1
3
2
4
3
2
3
6
3
4
3
6
5
2
2
6
3
2
6
6
3
6
6
3
6
3
6
6
6
6
2
6
6
6
6
6
6
6
6
6
6
6
6
6
6
1
6
3
6
6
6
6
6
6
4
6
6
6
6
6
6
1
6
6
6
6
3
6
1
6
6
6
2
3
6
6
4
6
5
6
4
6
6
6
6
6
3
2
2
6
6
3
5
6
2
5
6
6
6
6
6
6
6
4
6
6
4
4
6
6
6
6
6
6
6
4
2
6
6
6
6
6
6
3
6
6
6
6
1
1
1
2
6
6
1
1
6
1
1
6
6
6
6
6
6
6
6
6
1
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
2
6
6
6
1
6
2
6
6
6
1
6
6
6
6
6
2
6
6
6
6
6
6
6
6
6
6
6
6
2
6
6
6
6
6
6
4
6
6
2
6
4
2
6
5
6
5
6
3
1
6
1
1
3
4
6
4
6
6
6
1
3
2
6
4
2
6
2
6
6
2
6
2
1
1
1
6
4
6
6
3
6
6
6
6
6
1
2
2
2
2
6
6
2
1
5
3
6
2
6
2
2
6
6
6
6
3
3
6
6
6
3
2
2
5
6
5
2
2
2
2
2
2
2
2
2
6
5
3
6
2
6
6
2
6
6
6
6
6
6
1
6
6
6
2
6
2
6
6
6
6
6
6
6
6
6
2
6
4
3
3
1
1
6
2
6
6
6
6
6
6
6
6


6
6
6
6
6
6
6
2
2
3
6
3
3
2
2
2
2
2
2
2
6
1
6
6
6
6
6
6
6
6
6
2
2
6
6
6
6
2
6
1
6
1
2
6
6
1
6
6
6
6
4
2
2
2
2
6
2
1
3
3
6
2
4
5
4
6
6
6
6
1
1
1
2
1
4
6
4
6
4
6
6
6
6
6
2
6
3
6
6
1
3
2
6
3
2
4
3
6
4
2
6
3
6
3
6
3
5
3
6
1
1
5
6
1
6
2
4
2
6
1
1
1
1
1
2
1
3
1
1
1
6
1
3
5
2
4
6
4
5
6
6
6
4
6
4
4
3
5
4
6
2
2
6
5
3
4
1
6
2
6
5
2
3
6
3
6
5
6
6
6
6
6
6
6
3
6
6
6
6
4
3
4
5
6
6
3
5
2
1
2
3
2
2
2
6
2
2
2
2
3
2
6
6
2
6
6
6
3
6
5
6
1
6
3
6
6
6
5
5
5
1
4
6
1
2
2
3
3
3
4
2
2
3
1
1
3
1
2
6
1
6
4
6
3
5
6
6
6
6
6
6
6
6
2
1
6
1
1
2
6
1
3
6
6
6
2
6
6
6
6
3
2
5
5
6
6
5
6
5
3
3
2
4
6
6
6
6
2
6
6
5
6
6
5
6
5
3
6
5
1
1
2
2
3
1
2
2
6
6
6
6
6
6
3
1
2
6
2
2
6
6
3
4
6
1
6
3
6
6
6
6
4
6
3
4
1
3
1
3
1
2
6
6
6
6
5
5
5
3
3
6
6
1
3
2
2
2
6
6
6
1
2
2
6
2
6
2
2
6
1
3
1
1
1
1
1
6
6
6
6
1
1
6
5
3
1
6
6
6
6
2
2
2
6
6
2
3
6
3
6
4
6
6
6
6
6
6
6
6
6
6
6
6
6
6
4
4
4
4
4
4
4
4
4
4
4
4
6
4
3
3
3
4
4
3
4
4
3
3
4
5
3
3
3
3
3
3
3
3
3
3
3
6
6
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
6
6
3
6
3
3
6
3
3
4
5
6
6
3
6
1
3
5
5
4
1


...

1
2
5
2
1
6
1
3
3
6
6
3
3
6
1
2
6
6
1
1
6
3
6
6
4
3
4
6
6
1
3
6
6
4
6
4
1
3
2
1
6
3
3
6
6
2
6
6
6
3
6
1
3
2
2
3
2
6
2
3
1
3
6
5
6
6
6
6
1
3
1
3
3
6
2
2
6
6
2
2
3
6
2
6
4
2
6
2
1
6
6
6
5
6
6
1
5
1
6
6
6
2
4
6
6
3
4
3
2
6
2
6
3
3
2
6
2
2
1
4
6
1
6
2
4
6
1
1
3
4
3
6
2
6
6
2
1
3
3
6
3
6
1
2
6
6
4
1
4
3
6
2
4
2
6
3
6
2
1
2
1
6
4
4
4
2
6
2
1
6
2
1
3
6
3
3
6
2
3
3
6
1
6
6
1
3
2
6
2
1
3
1
2
3
1
3
6
3
1
2
2
6
4
3
6
3
6
6
1
6
2
6
6
1
3
2
6
6
6
4
1
5
1
3
5
2
6
6
6
2
1
6
5
2
6
2
6
3
2
6
6
2
2
6
2
6
1
1
2
2
6
1
2
4
2
6
1
1
6
6
2
2
2
1
6
6
6
2
6
6
1
6
2
2
6
6
6
3
1
4
4
2
5
6
1
4
3
1
5
6
4
6
1
1
1
6
3
2
6
1
2
4
3
1
6
1
3
2
2
2
6
1
3
1
3
6
2
6
1
4
1
1
2
6
3
3
6
1
6
1
1
6
3
3
2
6
5
1
2
2
3
5
5
2
2
6
2
6
2
6
3
3
2
2
3
6
1
6
6
6
1
6
3
2
1
6
3
6
6
2
1
6
3
6
2
6
6
6
3
2
1
4
2
6
1
1
6
2
6
2
1
2
1
6
6
4
6
6
2
6
2
1
6
6
2
6
2
2
6
2
2
6
6
6
6
2
2
6
6
2
6
6
6
2
2
6
2
6
6
6
6
3
1
6
6
6
2
6
4
2
2
1
2
6
6
1
2
2
6
1
6
6
4
5
2
4
2
1
1
6
6
6
5
3
3
1
2
1
1
5
1
6
1
6
2
6
1
1
6
6
6
3
6
6
2
1
1
6
6
5
1
5
1
2
6
2
6
6
4
1


6
1
6
1
1
4
1
2
1
1
4
1
3
1
1
2
3
2
4
2
6
6
1
1
3
6
6
5
6
3
2
1
6
2
6
1
2
1
6
3
1
1
4
3
2
1
1
1
3
4
1
1
3
1
2
2
5
2
3
1
1
3
2
1
2
4
1
2
6
1
1
6
1
2
1
1
6
2
2
1
2
1
6
6
5
1
1
5
6
3
3
2
3
1
1
2
6
1
2
6
1
1
1
1
2
3
1
1
1
2
1
2
6
1
2
1
1
2
3
2
1
6
1
3
1
3
5
2
1
3
2
1
2
2
6
6
6
2
1
2
1
1
2
2
2
5
1
2
6
2
1
1
3
5
1
1
3
3
6
1
1
2
6
1
6
6
1
1
2
2
1
1
2
1
2
1
6
4
2
1
1
2
3
4
2
1
1
1
1
1
6
3
6
1
1
1
4
4
6
1
1
6
4
1
2
1
1
1
5
4
6
5
1
5
1
1
4
6
1
1
6
2
5
6
1
1
4
5
1
3
1
6
1
1
5
1
1
5
6
6
6
5
6
2
3
3
6
5
6
6
1
5
3
1
4
3
6
1
5
1
6
3
3
5
6
1
6
5
3
4
5
6
3
5
6
1
4
6
3
5
2
2
1
6
2
6
3
3
3
6
1
2
6
3
6
3
6
2
1
3
6
1
1
3
3
6
6
3
2
3
6
1
6
6
6
6
1
6
3
6
2
6
3
4
2
6
4
4
6
4
6
3
6
1
1
6
3
4
6
5
1
2
3
4
6
1
2
3
3
2
4
6
6
2
6
1
6
3
3
3
5
2
4
2
3
6
2
3
3
2
1
6
6
2
2
6
6
6
3
2
3
1
2
6
6
3
4
6
6
2
5
6
6
3
5
2
6
3
6
6
6
3
6
2
2
6
6
4
3
6
6
2
2
3
6
6
1
6
6
6
3
3
6
6
6
6
3
6
2
6
2
2
4
2
4
1
3
6
4
2
6
2
3
3
6
6
2
4
6
6
2
2
6
2
3
6
4
3
6
1
1
2
3
4
6
1
3
6
1
1
4
3
2
1
4
3
2
4
4
2
3
6
1
4
2
1
3
3
6
6
3
4
6
3
1
6
4
4
4
3


2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
5
6
5
5
5
6
6
5
5
2
3
3
3
3
3
3
3
3
3
2
2
2
3
3
3
3
3
3
3
3
3
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
3
3
3
3
3
3
3
3
3
2
3
3
3
3
3
3
3
2
2
1
1
2
6
3
6
6
6
3
2
6
3
3
6
6
2
3
3
5
4
3
5
5
4
3
5
5
4
3
5
5
4
3
5
4
5
4
4
5
5
5
5
3
5
5
4
3
5
4
3
5
4
3
5
5
4
6
3
6
3
6
6
6
3
6
3
6
3
2
2
6
2
2
2
2
2
5
1
3
5
3
6
6
6
6
2
6
6
5
6
2
2
6
6
2
6
2
3
2
6
2
1
1
1
1
1
1
1
1
6
1
1
1
4
6
6
6
3
2
4
4
1
6
4
4
5
1
3
6
3
2
1
4
1
2
3
6
3
5
6
6
6
2
5
6
6
3
3
3
3
3
3
3
2
6
4
2
1
2
1
1
1
2
1
1
2
1
1
2
1
2
1
1
2
1
2
1
1
2
1
6
2
3
2
2
2
3
3
3
3
1
3
2
6
6
5
6
1
1
4
6
1
4
4
5
2
2
1
6
3
6
3
2
1
1
3
3
3
2
2
6
6
3
3
2
2
3
1
3
1
6
4
5
3
5
3
5
5
3
5
5
3
5
3
3
5
5
3
5
3
5
3
6
6
6
6
3
4
2
1
1
1
2
1
6
2
3
4
2
1
5
6
1
1
2
2
4
2
2
2
3
1
4
3
2
1
2
1
3
6
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
6
6
6
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
6
2
1
1
2
3
6
4
2
6
2
2
2
2
2
2
2
2
2
4
2
3
6
2
2
2
6
6
4
4
6
1
2
1
2
1
6
2
2
1
2
2
5
3
6
6
2
5
6
3
6
2
4
6
6
1
4
4
4
2
6
3
2
6
6
3
2
2
6
6
1
6


6
6
6
6
6
1
2
1
2
1
6
4
6
1
2
6
6
6
6
6
6
6
1
1
4
2
4
2
3
2
2
2
1
1
1
1
6
6
5
6
5
1
1
1
3
2
1
4
3
3
6
2
6
1
2
1
4
2
1
2
1
1
1
3
1
1
1
5
3
6
1
1
1
1
1
1
1
1
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
5
2
6
6
1
3
3
2
3
3
2
1
1
2
6
3
3
4
6
4
4
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
1
6
4
6
6
2
3
3
1
2
6
2
1
2
6
1
5
3
2
6
6
6
6
4
1
2
3
6
3
1
2
3
2
6
1
2
3
2
6
3
1
2
3
2
6
3
1
2
3
2
3
1
2
3
2
6
1
3
2
6
1
2
3
2
3
2
3
1
3
2
2
3
1
3
1
2
3
1
2
2
1
2
3
1
3
6
6
6
1
1
1
6
1
4
6
3
4
3
3
2
1
6
3
5
6
3
6
3
5
3
4
4
3
1
4
2
4
5
4
4
1
1
2
4
2
5
5
5
5
1
2
1
5
2
5
6
1
2
1
5
2
1
1
1
3
1
6
4
6
4
6
1
4
6
1
1
2
6
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
4
5
2
2
6
2
2
2
2
2
2
2
2
2
2
2
2
2
6
6
6
6
6
5
6
1
2
3


1        NaN
65       NaN
125      NaN
136      NaN
183      NaN
267      NaN
268      NaN
275      NaN
484      NaN
526      NaN
527      NaN
528      NaN
529      NaN
533      NaN
534      NaN
542      NaN
661      NaN
685      NaN
854      NaN
863      NaN
878      NaN
1031     NaN
1042     NaN
1049     NaN
1086     NaN
1398     NaN
1403     NaN
1404     NaN
1413     NaN
1498     NaN
          ..
120214   NaN
120224   NaN
120312   NaN
120450   NaN
120486   NaN
120523   NaN
120544   NaN
120552   NaN
120560   NaN
120562   NaN
120580   NaN
120583   NaN
120587   NaN
120601   NaN
120639   NaN
120641   NaN
120653   NaN
120655   NaN
120667   NaN
120668   NaN
120676   NaN
120687   NaN
120965   NaN
120979   NaN
121062   NaN
121069   NaN
121086   NaN
121101   NaN
121133   NaN
121134   NaN
Name: surface_total_in_m2, Length: 3991, dtype: float64